In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!nvidia-smi

Sun Nov  1 12:13:49 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
 from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
import torch
print(torch.__version__) 
torch.cuda.is_available()

1.6.0+cu101


True

In [5]:
! git clone https://github.com/NVIDIA/apex
os.chdir('apex')
! pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./
os.chdir('../')

Cloning into 'apex'...
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 7456 (delta 0), reused 0 (delta 0), pack-reused 7455
Receiving objects: 100% (7456/7456), 13.91 MiB | 26.48 MiB/s, done.
Resolving deltas: 100% (5036/5036), done.
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-vn2wd545
Created temporary directory: /tmp/pip-req-tracker-mga74cwf
Created requirements tracker '/tmp/pip-req-tracker-mga74cwf'
Created temporary directory: /tmp/pip-install-fws7e9ku
Processing /content/apex
  Created temporary directory: /tmp/pip-req-build-kk14oh1n
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-mga74cwf'
    Running setup.py (path:/tmp/pip-req-build-kk14oh1n/setup.py) egg_info 

In [6]:
#!pip install torch==1.5.0+cu101 torchvision==0.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install albumentations==0.4.6

     |████████████████████████████████| 122kB 10.9MB/s 
     |████████████████████████████████| 952kB 23.6MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-cp36-none-any.whl size=65165 sha256=eecafcf0dda4cea11dc7ad6f8602bae43b5ad5990012e89836398e1ad4df7e07
  Stored in directory: /root/.cache/pip/wheels/c7/f4/89/56d1bee5c421c36c1a951eeb4adcc32fbb82f5344c086efa14
Successfully built albumentations
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [7]:
import albumentations as A
from apex import amp

from albumentations.augmentations.transforms import RandomRain, RandomSnow, RandomFog
from albumentations.pytorch.transforms import ToTensorV2

In [8]:
import sys
#sys.path.insert(0, "/content/timm_efficientdet_pytorch")
#sys.path.insert(0, "/content/omegaconf")
#sys.path.insert(0, "/content/weightedboxesfusion")

import os
from datetime import datetime
import time
import random
import cv2
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler

from glob import glob
from skimage import io


np.random.seed(5)

In [9]:
#TRAIN_ROOT_PATH = '/content/drive/My Drive/Dhaka-AI 2020/dataset/train/'
IMG_SIZE = 512

In [10]:
#train_image_path = '/content/drive/My Drive/Dhaka-AI 2020/dataset/train/'
test_image_path = '/content/drive/My Drive/Dhaka-AI 2020/dataset/test/'

types = ('*.jpg','*.jpeg','*.png','*.JPG','*.PNG') # the tuple of file types


test_images = sorted(glob(test_image_path + '*'))



In [11]:
print(f' test images : {len(test_images)} items')


 test images : 500 items


In [12]:
from random import sample
IMG_SIZE = 512
DATA_ROOT_PATH = "/content/drive/My Drive/Dhaka-AI 2020/dataset/test/"
test_images = sorted(glob(DATA_ROOT_PATH + '*'))
from random import sample

def get_ids(path_list):
  path_list = random.sample(path_list,len(path_list))
  id_list = [path.split('/')[-1][:-4] for path in path_list]
  return np.array(id_list)

test_ids = get_ids(test_images)

len(test_ids)
print(test_ids)

['Shykat_4_ (59)_jpg.rf.baec676071651b7febaccebf0b7a5e74'
 'Shykat_5_ (47)_jpg.rf.6a9844ee2521d42cdf292c02c34054f0'
 'Shykat_02_051_jpg.rf.eb5dd25e6ca1768e793952d89c7d2298'
 'sabiha(231)_jpg.rf.ce0fa89f3da4d834e7ba276bc6c2d2b9'
 'Shykat_5_ (75)_jpg.rf.38b18bc2d862399072bd1b66cd942995'
 'sabiha(256)_jpg.rf.f72a769c9b64a31f3a0bf741fe495d4b'
 'Sabiha_(15)_jpg.rf.d57eca49cdf9b53adfeaf1c6564b6179'
 'sabiha(186)_jpg.rf.72bef88370058a83ed90b404283cef9b'
 'sabiha(160)_jpg.rf.9876842f206058208879845170d7b4e1'
 'sabiha(213)_jpg.rf.b7581c97308c651ac437829e21187f4b'
 'Shykat_4_ (33)_jpg.rf.c1ce8994e855d090ec1930b9eb013855'
 'Shykat_4_ (24)_jpg.rf.4f7b1d3e1be5e28dd863f370f8d025a8'
 'sabiha(252)_jpg.rf.00cd9a169d4de43b8967a68f02d2c12c'
 'sabiha(244)_jpg.rf.b987135ce5490e7d73429fa4688c1112'
 'Shykat_4_ (47)_jpg.rf.fa40bcaefb6887416db98b424ca9bf0a'
 'sabiha(156)_jpg.rf.a86f3be1fde683820d84cfc79d9fe2d2'
 'sabiha(141)_jpg.rf.a21ed07973ec474f7ab657480bc9f5f2'
 'Shykat_4_ (26)_jpg.rf.0667cc7161865a93baf6f

In [13]:
from sklearn import preprocessing
#getting the output files
outputfilepath = '/content/drive/My Drive/Dhaka-AI 2020/Code/sadia_effdet_test_result/'

In [14]:
#CHANGE HERE

outputfile = outputfilepath + 'test_result_effdet5_epoch37.csv'

marking_train = pd.read_csv(outputfile)
marking_train.head()

,image_id,class,score,xmin,ymin,xmax,ymax,width,height
0,Shykat_5_ (76)_jpg.rf.440799c009634a91033f13b2...,three wheelers (CNG),0.85274,700,584,770,712,1024,1024
1,Shykat_5_ (76)_jpg.rf.440799c009634a91033f13b2...,bus,0.72545,3,469,182,837,1024,1024
2,Shykat_5_ (76)_jpg.rf.440799c009634a91033f13b2...,bus,0.69342,302,414,349,496,1024,1024
3,Shykat_5_ (76)_jpg.rf.440799c009634a91033f13b2...,rickshaw,0.68819,359,553,404,712,1024,1024
4,Shykat_5_ (76)_jpg.rf.440799c009634a91033f13b2...,car,0.54383,574,529,661,656,1024,1024


In [15]:
#total classes
df = marking_train
print(df['class'].nunique())

print(df['class'].unique())

18
['three wheelers (CNG)' 'bus' 'rickshaw' 'car' 'van' 'minivan' 'suv'
 'motorbike' 'ambulance' 'bicycle' 'minibus' 'truck' 'pickup'
 'auto rickshaw' 'wheelbarrow' 'human hauler' 'army vehicle' 'taxi']


In [16]:
class_map = dict(zip(df['class'].unique(),list(range(1,22))))
print(df['class'].unique())
pd.DataFrame(df['class'].unique(),columns = ['class']).to_csv('class.csv',index=False)

df['classid'] = df['class'].apply(lambda x : class_map[x])
df[['xmin','ymin','xmax','ymax']] = df[['xmin','ymin','xmax','ymax']].astype(np.float)

['three wheelers (CNG)' 'bus' 'rickshaw' 'car' 'van' 'minivan' 'suv'
 'motorbike' 'ambulance' 'bicycle' 'minibus' 'truck' 'pickup'
 'auto rickshaw' 'wheelbarrow' 'human hauler' 'army vehicle' 'taxi']


In [17]:
df.head()

,image_id,class,score,xmin,ymin,xmax,ymax,width,height,classid
0,Shykat_5_ (76)_jpg.rf.440799c009634a91033f13b2...,three wheelers (CNG),0.85274,700.0,584.0,770.0,712.0,1024,1024,1
1,Shykat_5_ (76)_jpg.rf.440799c009634a91033f13b2...,bus,0.72545,3.0,469.0,182.0,837.0,1024,1024,2
2,Shykat_5_ (76)_jpg.rf.440799c009634a91033f13b2...,bus,0.69342,302.0,414.0,349.0,496.0,1024,1024,2
3,Shykat_5_ (76)_jpg.rf.440799c009634a91033f13b2...,rickshaw,0.68819,359.0,553.0,404.0,712.0,1024,1024,3
4,Shykat_5_ (76)_jpg.rf.440799c009634a91033f13b2...,car,0.54383,574.0,529.0,661.0,656.0,1024,1024,4


In [18]:
def get_test_transforms():
    return A.Compose([
            A.Resize(height=IMG_SIZE, width=IMG_SIZE, p=1.0),
            ToTensorV2(p=1.0),
        ], 
        p=1.0,
        bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels']
        ))

In [19]:
TEST_ROOT_PATH = test_image_path
class DatasetRetriever(Dataset):

    def __init__(self, marking, image_ids, transforms=None, test=False):
        super().__init__()

        self.image_ids = image_ids
        self.marking = marking
        self.transforms = transforms
        self.test = test

    def load_image_and_boxes(self, index):
        image_id = self.image_ids[index]
        image = cv2.imread(f'{TEST_ROOT_PATH}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32) 
        image /= 255.0
        records = self.marking[self.marking['image_id'] == image_id]
        boxes = records[['xmin', 'ymin', 'xmax', 'ymax']].values
        return image, boxes , records['classid']
    
    def __len__(self) -> int:
        return self.image_ids.shape[0]
    

    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        image, boxes, labels = self.load_image_and_boxes(index)
        
#         if self.test or random.random() > 0.5:
#             image, boxes = self.load_image_and_boxes(index)
#         else:
#             image, boxes = self.load_cutmix_image_and_boxes(index)
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['img_scale'] = torch.tensor([1.])
        target['image_id'] = torch.tensor([index])
        target['img_size'] = torch.tensor([(IMG_SIZE, IMG_SIZE)])

        try :
          if self.transforms:
            for i in range(10):
                sample = self.transforms(**{
                    'image': image,
                    'bboxes': target['boxes'],
                    'labels': target['labels']
                })
                if len(sample['bboxes']) > 0:
                    image = sample['image']
                    target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)
                    target['boxes'][:,[0,1,2,3]] = target['boxes'][:,[1,0,3,2]]  #yxyx: be warning
                    target['labels'] = torch.tensor(sample['labels'])
                return image, target, image_id
                print(i)
                    
        except:
          print("Error in this image: ",image_id)

        return image, target, image_id
      

    

      


test_dataset = DatasetRetriever(
    image_ids=test_ids,
    marking = df,
    transforms=get_test_transforms(),
    test = False
)




In [20]:
rows = 10

In [37]:
startindex = 291

In [38]:

fig=plt.figure(figsize=(100,100))
columns = 1

c_map = dict(zip(list(range(1,21)),pd.read_csv('class.csv')['class'].unique()))

for i in range(1, columns*rows +1):
    image,target, image_id = test_dataset[startindex]

    boxes = target['boxes'].cpu().numpy().astype(np.int32)
    classnames = target['labels'].cpu().numpy()
    numpy_image = image.permute(1,2,0).cpu().numpy()
    for j, box in enumerate(boxes):
        cv2.rectangle(numpy_image, (box[1], box[0]), (box[3],  box[2]), (0, 255, 0), 2)
        cv2.putText(numpy_image , c_map[classnames[j]] , (int(box[1]),int(box[0])), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    fig.add_subplot(rows, columns, i)
    plt.imshow(numpy_image)
    startindex += 1
plt.show()

Output hidden; open in https://colab.research.google.com to view.

Doesn't recognize any **police car, garbagevan, scooter**
# Image 201-300 :
## image 201-210 problems :


> 1.   Recognizes building as truck -- *solution : increase both truck and building images*
2.   Doesn't recognize far away vehicle, recognizes far away bus but bounding box is much larger than bus -- *solution : increase far away vehicle images*
3. 1 wrong bus, 1 wrong pickup (both not there)
4. Recognizes rickshaw colored fruit shop as rickshaw, detects actual rickshaw as wheelbarrow

## image 211-220 problems :


> 1.   detects faraway building as bus and truck 
2.   does not detect faraway car and suv (2)
3. doesn't detect many vehicles at night time
4. almost always detects wrong pickup and wrong wheelbarrow
5. when motorbike is up close, detects one motorbike as two
6. trouble detecting any faraway vehicle
7. detects one bus as two buses with much larger bounding box


## image 221-230 problems :


> 1.   does not detect faraway motorbike (3)
2.   when motorbike is up close, detects one motorbike as two (2)
3. detects one entire image as vehicle even though there's nothing there 

## image 231-240 problems :


> 1.   detects long bus as car, *need to increase long bus images*
2.   detects building as either bus or truck (mostly truck) (2) 

## image 241-250 problems :

> 1.  detects any structure other than vehicles as truck (3)
2.  always detects wrong pickup and human hauler (2)
3. when motorbike is up close, detects one motorbike as two (3)

## image 251-260 problems :

> 1.  detects any structure other than vehicles as truck (4)
2.  detects one entire image as vehicle even though there's nothing there (2)

The problems in images 261-300 are covered above.


*Observation :* 
*Seems like other than faraway vehicles, most of the vehicles are recognized although most of the time labelled wrong in case of pickups,wheelbarrows, human haulers. The model is detecting other structures as vehicles in a lot of images ( mostly as trucks ). Some vehicles are having two boxes ( up close motorbikes and buses ), one correct and one exceeding bound.*

*We need to increase confidence threshold ( a little ) and increase faraway vehicle images, wheelbarrow,pickup,up close motorbike,faraway and up close bus images. Also increasing some nighttime images might help. Also need to increase long bus images, increase images with large buildings and other structures and trucks.*

-Sadia












